# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.model import Model, InferenceConfig
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset, Datastore, Workspace

from azureml.pipeline.steps import AutoMLStep
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.environment import CondaDependencies

from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split
#from train import clean_data

from azureml.widgets import RunDetails

import joblib

print("SDK version: ", azureml.core.VERSION)



SDK version:  1.36.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print('My workspace name: ' + ws.name, 'My resource group: ' + ws.resource_group, 'Azure region: ' + ws.location, 'My sybscription id: ' + ws.subscription_id, sep = '\n')
# choose a name for experiment
my_experiment_name = 'my_automl_breast_cancer_experiment'

if "my_automl_project" not in os.listdir():
    os.mkdir("./my_automl_project")

my_automl_project_folder = './my_automl_project'

experiment=Experiment(ws, my_experiment_name)

My workspace name: quick-starts-ws-173266
My resource group: aml-quickstarts-173266
Azure region: southcentralus
My sybscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53


## Create an AMLCompute cluster

In [3]:
my_compute_cluster_name = "bc-cluster"

try:
    my_compute_target = ComputeTarget(workspace = ws, name = my_compute_cluster_name)
    print('An existing cluster has been found, please use it.')

except ComputeTargetException:
    my_compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes = 4, min_nodes = 1)
    my_compute_target = ComputeTarget.create(ws, my_compute_cluster_name, my_compute_config)

my_compute_target.wait_for_completion(show_output=True)
print(my_compute_target.get_status().serialize())

An existing cluster has been found, please use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-01-07T08:34:07.315000+00:00', 'errors': None, 'creationTime': '2022-01-07T08:26:48.895645+00:00', 'modifiedTime': '2022-01-07T08:26:54.573171+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


##  Access data

In [4]:
breast_cancer_url = 'https://raw.githubusercontent.com/JPSvdW/Machine_Learning_Engineer_with_Microsoft_Azure_Nanodegree_Capstone_Project/main/data/breast-cancer.csv'

breast_cancer_dataset = TabularDatasetFactory.from_delimited_files(breast_cancer_url)

breast_cancer_dataframe = breast_cancer_dataset.to_pandas_dataframe()

breast_cancer_dataframe.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
breast_cancer_dataframe.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


## Data cleaning

In [6]:
breast_cancer_dataframe["diagnosis"] = breast_cancer_dataframe.diagnosis.apply(lambda s: 1 if s == "M" else 0)
breast_cancer_dataframe = breast_cancer_dataframe.rename(columns={"diagnosis" : "malignant"})
breast_cancer_dataframe = breast_cancer_dataframe.drop(columns=["id"])
breast_cancer_dataframe = breast_cancer_dataframe.dropna()

#breast_cancer_dataframe.head()

my_train_data, my_test_data = train_test_split(breast_cancer_dataframe, test_size = 0.2, random_state = 40)

#print(my_train_data)

ws = Workspace.from_config()

my_datastore = ws.get_default_datastore()

final_training_data = Dataset.Tabular.register_pandas_dataframe(my_train_data, my_datastore, "automl_breast_cancer_dataset", show_progress = True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/107e0a72-0984-4381-9d56-45edfbf53a7a/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
automl_settings = {"experiment_timeout_minutes": 30, "max_concurrent_iterations": 4, "n_cross_validations": 5,
"primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = my_compute_target, task = "classification", training_data = final_training_data,
label_column_name = "malignant", path = my_automl_project_folder, enable_early_stopping = True, enable_onnx_compatible_models = True,
featurization = 'auto', debug_log = "automl_errors.log", **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(config = automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
my_automl_breast_cancer_experiment,AutoML_6f2f8462-51c7-4eab-aaf4-09a54195d02a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
my_automl_breast_cancer_experiment,AutoML_6f2f8462-51c7-4eab-aaf4-09a54195d02a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_6f2f8462-51c7-4eab-aaf4-09a54195d02a',
 'target': 'bc-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-07T08:49:42.858922Z',
 'endTimeUtc': '2022-01-07T09:10:00.965902Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'bc-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"my_automl_breast_cancer_experiment","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-173266","workspace_name":"quick-starts-ws-173266","region":"southcentralus","compute_target

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
my_automl_best_run, my_automl_fitted_model = remote_run.get_output()

In [11]:
my_automl_best_run.get_metrics()

{'recall_score_weighted': 0.9846153846153847,
 'matthews_correlation': 0.9679300351669979,
 'f1_score_macro': 0.9836679274996353,
 'balanced_accuracy': 0.9823581419169927,
 'f1_score_micro': 0.9846153846153847,
 'precision_score_micro': 0.9846153846153847,
 'average_precision_score_micro': 0.9955865347468984,
 'log_loss': 0.10089197129369873,
 'f1_score_weighted': 0.9845550845297723,
 'precision_score_weighted': 0.9850558005776561,
 'AUC_micro': 0.995628547276899,
 'AUC_weighted': 0.9953108639901185,
 'recall_score_macro': 0.9823581419169927,
 'average_precision_score_weighted': 0.9958127160399926,
 'average_precision_score_macro': 0.9956538805526088,
 'norm_macro_recall': 0.9647162838339852,
 'accuracy': 0.9846153846153847,
 'recall_score_micro': 0.9846153846153847,
 'precision_score_macro': 0.9856121938418742,
 'weighted_accuracy': 0.9866808101418492,
 'AUC_macro': 0.9953108639901183,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_6f2f8462-51c7-4eab-aaf4-09a54195d02a_

In [12]:
print(my_automl_best_run)

Run(Experiment: my_automl_breast_cancer_experiment,
Id: AutoML_6f2f8462-51c7-4eab-aaf4-09a54195d02a_36,
Type: azureml.scriptrun,
Status: Completed)


In [13]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' _ ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' _ ')
        else:
            pprint(step[1].get_params())
            print()

print_model(my_automl_fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': False,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': True,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/breast-cancer-compute/code/Users/odl_user_173266'}

prefittedsoftvotingclassifier
{'estimators': ['17', '15', '20', '14', '23', '10', '27', '9'],
 'weights': [0.15384615384615385,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.23076923076923078]}

17 _ maxabsscaler
{'copy': True}

17 _ logisticregression
{'C': 1.7575106248547894,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'm

In [14]:
print(my_automl_fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('svcwrapper', SVCWrapper(C=16.768329368110066, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001, verbose=False))], verbose=False))], flatten_transform=None, weights=[0.15384615384615385, 0.07692307692307693, 0.15384615384615385, 0.07692307692307693, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.23076923076923078]))],
         verbose=False)


In [15]:
os.makedirs('outputs', exist_ok = True)

joblib.dump(value = my_automl_fitted_model, filename = 'outputs/my-best-automl-model.pkl') ## add this model name to score.py file

['outputs/my-best-automl-model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
my_best_automl_registered_model = Model.register(workspace = ws, model_path = 'outputs/my-best-automl-model.pkl', model_name = 'my-best-automl-model')

my_automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'my-best-model-conda-env.yml')

Registering model my-best-automl-model


In [17]:
my_automl_best_run.download_file("outputs/scoring_file_v_2_0_0.py", "score.py")

In [18]:
env = Environment.from_conda_specification('my-best-model-conda-env', 'my-best-model-conda-env.yml')

inference_config = InferenceConfig(entry_script = './score.py', environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights = True,
tags = {'name':'Breast cancer prediction'}, description = 'This is my breast cancer prediction model')

service = my_best_automl_registered_model.deploy(workspace = ws, name = 'breast-cancer-prediction', models = [my_best_automl_registered_model],
inference_config = inference_config, deployment_config = aci_config)

service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-07 09:18:56+00:00 Creating Container Registry if not exists..
2022-01-07 09:28:56+00:00 Registering the environment.
2022-01-07 09:28:58+00:00 Building image..
2022-01-07 09:40:57+00:00 Generating deployment configuration.
2022-01-07 09:40:58+00:00 Submitting deployment to compute.
2022-01-07 09:41:02+00:00 Checking the status of deployment breast-cancer-prediction..
2022-01-07 09:43:32+00:00 Checking the status of inference endpoint breast-cancer-prediction.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [65]:
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
print("swagger URI: " + service.swagger_uri)

swagger URI: http://35f98038-761a-40e5-bd44-c56af74fdb54.southcentralus.azurecontainer.io/swagger.json


## Send a request tothe deployed web service

In [66]:
import requests
import json

scoringuri = service.scoring_uri

key = service.get_keys()[0]

#print(scoringuri)
#print(key)

data = {
  "Inputs": {
    "data": [
      {
        "radius_mean": 18.0,
        "texture_mean": 10.40,
        "perimeter_mean": 122.9,
        "area_mean": 1002,
        "smoothness_mean": 0.1185,
        "compactness_mean": 0.2778,
        "concavity_mean": 0.3002,
        "concave points_mean": 0.1473,
        "symmetry_mean": 0.2420,
        "fractal_dimension_mean": 0.07873,
        "radius_se": 1.096,
        "texture_se": 0.9054,
        "perimeter_se": 8.590,
        "area_se": 153.6,
        "smoothness_se": 0.00640,
        "compactness_se": 0.04906,
        "concavity_se": 0.05375,
        "concave points_se": 0.01588,
        "symmetry_se": 0.03004,
        "fractal_dimension_se": 0.006195,
        "radius_worst": 25.40,
        "texture_worst": 17.40,
        "perimeter_worst": 184.8,
        "area_worst": 2020,
        "smoothness_worst": 0.1625,
        "compactness_worst": 0.6658,
        "concavity_worst": 0.7120,
        "concave points_worst": 0.2656,
        "symmetry_worst": 0.4603,
        "fractal_dimension_worst": 0.1190
      },
      {
        "radius_mean": 13.56,
        "texture_mean": 14.37,
        "perimeter_mean": 87.50,
        "area_mean": 567.0,
        "smoothness_mean": 0.09778,
        "compactness_mean": 0.08130,
        "concavity_mean": 0.06666,
        "concave points_mean": 0.04783,
        "symmetry_mean": 0.1886,
        "fractal_dimension_mean": 0.05768,
        "radius_se": 0.2700,
        "texture_se": 0.7888,
        "perimeter_se": 2.060,
        "area_se": 24.0,
        "smoothness_se": 0.008470,
        "compactness_se": 0.0147,
        "concavity_se": 0.02388,
        "concave points_se": 0.01316,
        "symmetry_se": 0.0199,
        "fractal_dimension_se": 0.0024,
        "radius_worst": 15.12,
        "texture_worst": 19.30,
        "perimeter_worst": 99.9,
        "area_worst": 712.0,
        "smoothness_worst": 0.145,
        "compactness_worst": 0.1775,
        "concavity_worst": 0.240,
        "concave points_worst": 0.1289,
        "symmetry_worst": 0.2979,
        "fractal_dimension_worst": 0.07260
      }
    ]
  },
  "method": "predict"
}
input_data = json.dumps(data)
#print(input_data)
with open("data.json", "w") as _f:
    _f.write(input_data)


headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

response = requests.post(scoringuri, input_data, headers=headers)
print(response.json())

{'Results': [1, 0]}


## Print webservice logs

In [64]:
print(service.get_logs())

2022-01-07T09:43:19,506905900+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-07T09:43:19,506906000+00:00 - rsyslog/run 
2022-01-07T09:43:19,509535000+00:00 - iot-server/run 
2022-01-07T09:43:19,598133400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_bec455c772c2d018a0dc9c15319d853a/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-07T09:43:20,382818300+00:00 - iot-server/finish 1 0
2022-01-07T09:43:20,384957600+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (70)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 101
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-01-07 09:43:23,017 | root | INFO | Starting up app insights client
logging socket was

## Convert to ONNX format and save

In [62]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

my_automl_best_run_onnx_model, my_automl_fited_onnx_model = remote_run.get_output(return_onnx_model = True)

OnnxConverter.save_onnx_model(my_automl_fited_onnx_model, './outputs/my_best_automl_model.onnx')

TODO: In the cell below, print the logs of the web service and delete the service

## Delete the service

In [67]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
